In [1]:
import pandas as pd
import plotly.express as px
import spotipy
import spotipy.oauth2 as oauth2
import plotly.graph_objects as go

In [2]:
# Setup credentials 

client_id = "9373835c9a8b419d963b19ad44c38894"
client_secret = "b492ec63fa0346fda22aa631abc3be72"

In [3]:
credentials = oauth2.SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret)

token = credentials.get_access_token()
sp = spotipy.Spotify(auth=token)
sp.trace = False

/Users/isaaclopez/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning:

You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.



In [4]:
#exsample of spotify desktop ID 
urn = "spotify:track:2TH65lNHgvLxCKXM3apjxI"
urn2 = "spotify:track:38RIEav39FGi86jppxO0du"

In [5]:
#returns a dataframe of a spotify track's data
def track_info(urn):
    #retrieve track info from api, spoitify documentaion
    track_feat = sp.audio_features(tracks=urn)
    track_popu = sp.track(urn)
    
    # code to flatten lit of dictionary returned 
    dj_feat = {} 
    for f in track_feat: 
        dj_feat.update(f)
    
    #remove unnecessary dictionary keys
    keys_to_remove = ["key","mode","speechiness","acousticness",
        "instrumentalness","tempo","type","id","uri",
        "track_href","analysis_url","duration_ms","time_signature","loudness"]
    for _ in keys_to_remove:
        del dj_feat[_]
    
    #extract track popularit from api return dictionary
    popularity = dict((k, track_popu[k]) for k in ["popularity"] if k in track_popu)
    
    #concatinate both dictionaries
    dj_feat.update(popularity)
    
    #normalize popularity in dictionary
    dj_feat["popularity"] = dj_feat['popularity']/100
    
    #create and transpose dataframe
    return pd.DataFrame.from_dict(dj_feat,orient="index").T

In [8]:
def make_radar(urn_s, urn_p):
    #create and save data frame
    searched = track_info(urn_s)
    predicted = track_info(urn_p)
    #feature labels
    categories = list(searched)
    #store track values
    s_values = searched.iloc[0].tolist()
    p_values = predicted.iloc[0].tolist()
    
    fig = go.Figure()
    
    #section for searched song
    fig.add_trace(go.Scatterpolar(
          r=s_values,
          theta=categories,
          fill='toself',
          line=dict(color="#ff007f"),
          name='searched track'
    ))
    #section for predicted song
    fig.add_trace(go.Scatterpolar(
          r=p_values,
          theta=categories,
          fill='toself',
          line=dict(color="#6a0dad"),
          name='Recommanded track'
    ))

    fig.update_layout(
      template = "plotly_dark",
      polar=dict(
        radialaxis=dict(
          visible=True,
        )),
      showlegend=False
    )

    fig.show()
    

In [9]:
make_radar(urn,urn2)